<a href="https://colab.research.google.com/github/Shreyas188/Autonomous-Driving-Lane-Obstacle-YOLOv8-Detection/blob/main/Autonomous_Driving_Lane_%2B_Obstacle_(YOLOv8)_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ultralytics opencv-python-headless numpy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 23.5 MB/s eta 0:00:00


In [2]:
import cv2
import numpy as np
from ultralytics import YOLO
from google.colab.patches import cv2_imshow


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
model = YOLO("yolov8s.pt")   # small fast model


In [4]:
from google.colab import files
uploaded = files.upload()

video_path = list(uploaded.keys())[0]
print("Video Uploaded:", video_path)


Saving WhatsApp Video 2025-12-16 at 11.22.06.mp4 to WhatsApp Video 2025-12-16 at 11.22.06.mp4
Video Uploaded: WhatsApp Video 2025-12-16 at 11.22.06.mp4


In [5]:
def detect_lanes(frame):

    height, width = frame.shape[:2]

    # 1. Convert to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # 2. Reduce noise
    blur = cv2.GaussianBlur(gray, (5, 5), 0)

    # 3. Edge detection
    edges = cv2.Canny(blur, 50, 150)

    # 4. Region of Interest (trapezoid area)
    mask = np.zeros_like(edges)
    polygon = np.array([
        [(0, height),
         (width, height),
         (int(width * 0.55), int(height * 0.6)),
         (int(width * 0.45), int(height * 0.6))]
    ])

    cv2.fillPoly(mask, polygon, 255)
    roi = cv2.bitwise_and(edges, mask)

    # 5. Hough Transform
    lines = cv2.HoughLinesP(
        roi,
        rho=1,
        theta=np.pi / 180,
        threshold=40,
        minLineLength=50,
        maxLineGap=120,
    )

    # Separate left and right lines
    left_lines = []
    right_lines = []

    if lines is not None:
        for x1, y1, x2, y2 in lines[:, 0]:
            if x2 == x1:
                continue

            slope = (y2 - y1) / (x2 - x1)

            if abs(slope) < 0.5:
                continue

            if slope < 0:
                left_lines.append((x1, y1, x2, y2))
            else:
                right_lines.append((x1, y1, x2, y2))

    # Draw averaged lane line
    def draw_avg_line(lines, color):
        if len(lines) == 0:
            return

        x_vals = []
        y_vals = []
        for x1, y1, x2, y2 in lines:
            x_vals.extend([x1, x2])
            y_vals.extend([y1, y2])

        if len(x_vals) < 2:
            return

        poly = np.polyfit(x_vals, y_vals, 1)
        slope, intercept = poly

        y1 = height
        y2 = int(height * 0.6)
        x1 = int((y1 - intercept) / slope)
        x2 = int((y2 - intercept) / slope)

        cv2.line(frame, (x1, y1), (x2, y2), color, 8)

    draw_avg_line(left_lines, (0, 255, 0))
    draw_avg_line(right_lines, (0, 255, 0))

    return frame


In [6]:
cap = cv2.VideoCapture(video_path)

fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter("output_result.mp4", fourcc, 30.0,
                      (int(cap.get(3)), int(cap.get(4))))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # 1. Lane detection
    lane_frame = detect_lanes(frame.copy())

    # 2. YOLO object detection
    results = model.predict(frame, conf=0.45)

    for r in results:
        for box in r.boxes:
            x1, y1, x2, y2 = box.xyxy[0]
            cls = int(box.cls[0])
            conf = float(box.conf[0])
            label = model.names[cls]

            cv2.rectangle(lane_frame, (int(x1), int(y1)),
                                       (int(x2), int(y2)),
                                       (255, 0, 0), 2)
            cv2.putText(lane_frame, f"{label} {conf:.2f}",
                        (int(x1), int(y1)-5),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.6, (255, 0, 0), 2)

    out.write(lane_frame)

cap.release()
out.release()

print("Processing Completed! Saved as output_result.mp4")


Streaming output truncated to the last 5000 lines.
Speed: 1.1ms preprocess, 7.6ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 7.6ms
Speed: 1.2ms preprocess, 7.6ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 7.5ms
Speed: 1.1ms preprocess, 7.5ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 6.9ms
Speed: 1.1ms preprocess, 6.9ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 7.0ms
Speed: 1.1ms preprocess, 7.0ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 8.5ms
Speed: 1.2ms preprocess, 8.5ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 8.2ms
Speed: 1.2ms preprocess, 8.2ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 8.0ms
Speed: 1.2ms preprocess, 8.0ms i

In [7]:
from google.colab import files
files.download("output_result.mp4")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>